In [1]:
class IO():
    def __init__(self, input_=[]):
        self.input_buffer = input_.copy()
        self.output_buffer = []
    
    def add_input(self, value, map_fn=int):
        for c in value:
            self.input_buffer.append(map_fn(c))
        return self

    def read(self):
        return self.input_buffer.pop(0)
    
    def write(self, value):
        self.output_buffer.append(value)
        return self
    
    def output(self):
        return self.output_buffer

In [2]:
import advent
from advent.intcode import run
import numpy as np

data = advent.get_intcode(19)

res = np.full((50, 50), '.')
print_np = lambda res: print('\n'.join([''.join(line) for line in res]))

count = 0
for i in range(50):
    for j in range(50):
        _, out = run(data.copy(), IO([i, j]))
        if out[0] == 1:
            count += 1
        res[(i, j)] = ('#' if out[0] else '.')

print(count)

186


In [9]:
def getter(x, y, data):
    if x < 0 or y < 0: return 0
    _, out = run(data.copy(), IO([y, x]))
    return out[0]

def attempt_grid(getter, size=3, y=5):
    # on the line with coordinates (x+size, :),
    # attempts to find the first '#'
    # then from (x,y) and see if you find enough '#' to complete the square
    # e.g. attempt_grid(3, 20) = False, attempt_grid(3, 21) = True
    x = 0
    while True:
        if getter(x, y+size-1) == 1: break
        x += 1
    res = getter(x+size-1, y) == 1 and getter(x+size-1, y+size-1) == 1 and getter(x, y) == 1
    #res = True
    #for i in range(size):
    #    out = getter(x, y+i)
    #    if out == 0: res=False
    print(f"Now attempting size={size}, y={y}, found x={x}, size={size}, xy={str(x*10000 + y)}. Result: {res}")
    print(f"Corners: {(x, y)}:{getter(x, y)}, {(x+size-1, y)}:{getter(x+size-1, y)}, {(x, y+size-1)}:{getter(x, y+size-1)}, {(x+size-1, y+size-1)}:{getter(x+size-1, y+size-1)}")
    return res

def binary_search(getter, size = 100, min_ = 10, max_ = 1200):
    # Find the minimum x for which attempt_grid(size, x) = True
    # assume attempt_grid(min_, x) = False and attempt_grid(max_, x) = True
    if (max_ - min_) <= 1:
        attempt_grid(getter, size, max_-3)
        attempt_grid(getter, size, max_-2)
        attempt_grid(getter, size, max_-1)
        assert attempt_grid(getter, size, max_)
        return max_
    next = (min_ + max_) // 2
    if attempt_grid(getter, size, next):
        return binary_search(getter, size, min_, next)
    return binary_search(getter, size, next, max_)


In [10]:
binary_search(lambda x, y: getter(x, y, data.copy()), 100)

Now attempting size=100, y=605, found x=786, size=100, xy=7860605. Result: False
Corners: (786, 605):1, (885, 605):0, (786, 704):1, (885, 704):1
Now attempting size=100, y=902, found x=1118, size=100, xy=11180902. Result: False
Corners: (1118, 902):1, (1217, 902):0, (1118, 1001):1, (1217, 1001):1
Now attempting size=100, y=1051, found x=1284, size=100, xy=12841051. Result: True
Corners: (1284, 1051):1, (1383, 1051):1, (1284, 1150):1, (1383, 1150):1
Now attempting size=100, y=976, found x=1201, size=100, xy=12010976. Result: True
Corners: (1201, 976):1, (1300, 976):1, (1201, 1075):1, (1300, 1075):1
Now attempting size=100, y=939, found x=1159, size=100, xy=11590939. Result: True
Corners: (1159, 939):1, (1258, 939):1, (1159, 1038):1, (1258, 1038):1
Now attempting size=100, y=920, found x=1138, size=100, xy=11380920. Result: False
Corners: (1138, 920):1, (1237, 920):0, (1138, 1019):1, (1237, 1019):1
Now attempting size=100, y=929, found x=1148, size=100, xy=11480929. Result: True
Corners:

926

In [91]:
# This is what I used for debugging, in combination with print_np(res) to see when the first grid of size 3 would occur
binary_search(lambda x, y: getter(x, y, data.copy()), 3, min_=15, max_=35)

Now attempting size=3, x=21, found y=25, size=3, xy=250021. Result: False
Corners: (21, 25):1, (23, 25):0, (21, 27):1, (23, 27):1
Now attempting size=3, x=24, found y=30, size=3, xy=300024. Result: True
Corners: (24, 30):1, (26, 30):1, (24, 32):1, (26, 32):1
Now attempting size=3, x=22, found y=27, size=3, xy=270022. Result: True
Corners: (22, 27):1, (24, 27):1, (22, 29):1, (24, 29):1
Now attempting size=3, x=21, found y=26, size=3, xy=260021. Result: True
Corners: (21, 26):1, (23, 26):1, (21, 28):1, (23, 28):1
Now attempting size=3, x=19, found y=23, size=3, xy=230019. Result: False
Corners: (19, 23):1, (21, 23):0, (19, 25):1, (21, 25):1
Now attempting size=3, x=20, found y=24, size=3, xy=240020. Result: False
Corners: (20, 24):1, (22, 24):0, (20, 26):1, (22, 26):1
Now attempting size=3, x=21, found y=25, size=3, xy=250021. Result: False
Corners: (21, 25):1, (23, 25):0, (21, 27):1, (23, 27):1
Now attempting size=3, x=21, found y=26, size=3, xy=260021. Result: True
Corners: (21, 26):1,

26

In [92]:
# Wrong guesses:
# 11450926
# 9261145

In [102]:
small = advent.get_lines('19small2')
def getter_small(x, y):
    #print(x, y)
    if x < 0 or y < 0: return 0
    if x >= 50 or y >= 50: return 0
    return 0 if small[y][x] == '.' else 1

# This is what I used for debugging, in combination with print_np(res) to see when the first grid of size 3 would occur
binary_search(getter_small, 10, min_=15, max_=30)

Now attempting size=10, x=26, found y=22, size=10, xy=260022. Result: True
Corners: (26, 22):1, (35, 22):1, (26, 31):1, (35, 31):1
Now attempting size=10, x=23, found y=18, size=10, xy=230018. Result: False
Corners: (23, 18):1, (32, 18):0, (23, 27):1, (32, 27):1
Now attempting size=10, x=25, found y=20, size=10, xy=250020. Result: True
Corners: (25, 20):1, (34, 20):1, (25, 29):1, (34, 29):1
Now attempting size=10, x=24, found y=19, size=10, xy=240019. Result: False
Corners: (24, 19):1, (33, 19):0, (24, 28):1, (33, 28):1
Now attempting size=10, x=22, found y=17, size=10, xy=220017. Result: False
Corners: (22, 17):1, (31, 17):0, (22, 26):1, (31, 26):1
Now attempting size=10, x=23, found y=18, size=10, xy=230018. Result: False
Corners: (23, 18):1, (32, 18):0, (23, 27):1, (32, 27):1
Now attempting size=10, x=24, found y=19, size=10, xy=240019. Result: False
Corners: (24, 19):1, (33, 19):0, (24, 28):1, (33, 28):1
Now attempting size=10, x=25, found y=20, size=10, xy=250020. Result: True
Cor

20

In [98]:
small[26]

'..............................#####...............'